## Exercise 1:  Wide and Deep Networks

- Try changing your assumptions about which features should be considered by the "non-linear" (deep) part of the network.  Can you get any improvements?
- Try using a different activation function in your wide and deep network. Do you get any improvements here?

In [30]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)
# Reserve an item for prediction
X_new = X_test[:3]
import numpy as np
# Note the offsets are used here to split up the features according to our input layer
X_train_wide, X_train_deep = X_train[:, 2:4], np.delete(X_train, np.s_[2:4], axis=1)
X_valid_wide, X_valid_deep = X_valid[:, 2:4], np.delete(X_valid, np.s_[2:4], axis=1)
X_test_wide, X_test_deep = X_test[:, 2:4], np.delete(X_test, np.s_[2:4], axis=1)
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

input_wide = tf.keras.layers.Input(shape=[2])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6])  # features 2 to 7
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30,activation='softmax')(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="softmax")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

# Remember to adapt the normalization layers!!!
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 4.1701 - dense_2_loss: 4.1430 - dense_3_loss: 4.4140 - dense_2_root_mean_squared_error: 2.0354 - dense_3_root_mean_squared_error: 2.1009 - val_loss: 2.7987 - val_dense_2_loss: 2.7598 - val_dense_3_loss: 3.1489 - val_dense_2_root_mean_squared_error: 1.6613 - val_dense_3_root_mean_squared_error: 1.7745
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 2.2248 - dense_2_loss: 2.1905 - dense_3_loss: 2.5334 - dense_2_root_mean_squared_error: 1.4800 - dense_3_root_mean_squared_error: 1.5917 - val_loss: 1.6084 - val_dense_2_loss: 1.5833 - val_dense_3_loss: 1.8346 - val_dense_2_root_mean_squared_error: 1.2583 - val_dense_3_root_mean_squared_error: 1.3545
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 1.4449 - dense_2_loss: 1.4273 - dense_3_loss: 1.6032 - dense_2_root_mean_squared_error: 1.1947 - dense_3_root_mean_squared_error: 1.2662 - val_loss: 1.2130 - val_dense_2_loss

In [15]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
print(f"""
Weighed Sum of Losses: {weighted_sum_of_losses}
Main Loss: {main_loss}
Aux Loss: {aux_loss}
Main RMSE: {main_rmse}
Aux RMSE: {aux_rmse}
      """)

162/162 [==============================] - 0s 1ms/step - loss: 0.3498 - dense_2_loss: 0.3495 - dense_3_loss: 0.3527 - dense_2_root_mean_squared_error: 0.5911 - dense_3_root_mean_squared_error: 0.5939

Weighed Sum of Losses: 0.3497806787490845
Main Loss: 0.3494572341442108
Aux Loss: 0.3526919484138489
Main RMSE: 0.591149091720581
Aux RMSE: 0.5938787460327148
      


## Exercise 2: Modifying the Wide and Deep Network

1. Modify the constructor of the wide and deep network to take parameters that govern the number of hidden layers and their width.  Now try running the new network with different properties for your hidden layers.  Can you improve your performance?

2. Modify the network class to use BatchNormalization, but add normalization after the activation function.  Examine your performance.

3. Modify the preceding class to use BatchNormalization before the activation function.  Examine your performance. 

In [31]:
import tensorflow as tf

tf.keras.backend.clear_session()
tf.random.set_seed(42)

input_wide = tf.keras.layers.Input(shape=[2])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6])  # features 2 to 7

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

norm_deep_normalised=tf.keras.layers.BatchNormalization()(norm_deep)
hidden1 = tf.keras.layers.Dense(30, activation='softmax')(norm_deep_normalised)
hidden1_normalized = tf.keras.layers.BatchNormalization()(hidden1)

hidden2 = tf.keras.layers.Dense(30, activation="softmax")(hidden1_normalized)
hidden2_normalized = tf.keras.layers.BatchNormalization()(hidden2)

concat = tf.keras.layers.concatenate([norm_wide, hidden2_normalized])

output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2_normalized)

model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

# Remember to adapt the normalization layers!!!
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.7640 - dense_2_loss: 1.6491 - dense_3_loss: 2.7978 - dense_2_root_mean_squared_error: 1.2842 - dense_3_root_mean_squared_error: 1.6727 - val_loss: 1.2352 - val_dense_2_loss: 1.2252 - val_dense_3_loss: 1.3246 - val_dense_2_root_mean_squared_error: 1.1069 - val_dense_3_root_mean_squared_error: 1.1509
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5183 - dense_2_loss: 0.5218 - dense_3_loss: 0.4866 - dense_2_root_mean_squared_error: 0.7224 - dense_3_root_mean_squared_error: 0.6976 - val_loss: 0.4963 - val_dense_2_loss: 0.4982 - val_dense_3_loss: 0.4793 - val_dense_2_root_mean_squared_error: 0.7058 - val_dense_3_root_mean_squared_error: 0.6923
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4611 - dense_2_loss: 0.4635 - dense_3_loss: 0.4397 - dense_2_root_mean_squared_error: 0.6808 - dense_3_root_mean_squared_error: 0.6631 - val_loss: 0.4072 - val_dense_2_loss

In [32]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

print(f"""
Weighed Sum of Losses: {weighted_sum_of_losses}
Main Loss: {main_loss}
Aux Loss: {aux_loss}
Main RMSE: {main_rmse}
Aux RMSE: {aux_rmse}
""")

162/162 [==============================] - 0s 974us/step - loss: 0.3435 - dense_2_loss: 0.3425 - dense_3_loss: 0.3531 - dense_2_root_mean_squared_error: 0.5852 - dense_3_root_mean_squared_error: 0.5942

Weighed Sum of Losses: 0.34352004528045654
Main Loss: 0.34245535731315613
Aux Loss: 0.35310137271881104
Main RMSE: 0.585196852684021
Aux RMSE: 0.5942233204841614



## Exercise 3: Comparing SELU and ReLU

Let's examine the performance of SELU and ReLU in the context of a deep network with 100 layers.  I've set up the example below to use SELU and Lecun Normalization.  Examine the performance, and then contrast using ReLU and He normalization.

In [20]:
import tensorflow as tf
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))

# 100 hidden layers
for layer in range(100):
    # model.add(tf.keras.layers.Dense(100, activation="selu",
    #                                 kernel_initializer="lecun_normal"))
    model.add(tf.keras.layers.Dense(100, activation="relu",
                                    kernel_initializer="he_normal"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.001),
              metrics=["accuracy"])

Get the data and train the network:

In [23]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

# Remember to scale the inputs!
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 18s 9ms/step - loss: 2.0980 - accuracy: 0.1969 - val_loss: 1.8966 - val_accuracy: 0.2602
Epoch 2/5
1719/1719 [==============================] - 15s 9ms/step - loss: 1.7103 - accuracy: 0.3469 - val_loss: 1.5307 - val_accuracy: 0.4194
Epoch 3/5
1719/1719 [==============================] - 15s 9ms/step - loss: 1.3293 - accuracy: 0.4725 - val_loss: 1.1189 - val_accuracy: 0.5244
Epoch 4/5
1719/1719 [==============================] - 16s 9ms/step - loss: 1.1417 - accuracy: 0.5395 - val_loss: 1.0240 - val_accuracy: 0.6002
Epoch 5/5
1719/1719 [==============================] - 16s 9ms/step - loss: 1.0020 - accuracy: 0.6135 - val_loss: 1.1292 - val_accuracy: 0.5320


In [24]:
eval_results = model.evaluate(X_test,y_test)
eval_results

313/313 [==============================] - 1s 3ms/step - loss: 3.5996 - accuracy: 0.0896


[3.599595785140991, 0.08959999680519104]

In [26]:
eval_results_relu

[3.599595785140991, 0.08959999680519104]

In [22]:
eval_results_selu


[1.484071135520935, 0.5303999781608582]